In [17]:
import pandas as pd
import re
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [19]:
df = pd.read_csv('master.csv')
df = df.iloc[2000:5151]
df = df[["Coding","Citation"]]
df = df[pd.to_numeric(df["Coding"], errors="coerce").notna()]
df["Coding"] = df["Coding"].astype(int) 
df.shape

/var/folders/gt/3sb0875538n39l60gwltp88w0000gn/T/ipykernel_46201/281173097.py:1: DtypeWarning: Columns (1,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('master.csv')


(1629, 2)

In [21]:
df = df[df["Citation"].str.split().str.len() > 5]
df["Citation"] = df["Citation"].astype(str) 
df.shape

(1623, 2)

In [23]:
print(df.dtypes)
print(df.head())

Coding       int64
Citation    object
dtype: object
      Coding                                           Citation
2000       2   Some of the p rimary policing institutions th...
2001       2   Sandy was prepared for him and intended to ki...
2002       3   In this book Franklin also states, “The episo...
2003       2   In  “Slave Patrols ” Kaiser explains, “ Patro...
2004       2   The initial LAPD rep orts say the cops had hi...


In [25]:
label_map = {
    1: "Background",
    2: "Exhibit",
    3: "Argument",
    4: "Methods",
    5: "Definition"  
}

X = df["Citation"]
y = df["Coding"]

In [35]:
df = df.rename(columns={"Coding": "label", "Citation": "text"})
df = df[["text", "label"]].dropna()

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

num_labels = df["label"].nunique()
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./bert_citation_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)
trainer.train()

Map:   0%|          | 0/1298 [00:00<?, ? examples/s]

Map:   0%|          | 0/325 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,0.495524
2,No log,0.534977
3,No log,0.767873


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=489, training_loss=0.40751098757628773, metrics={'train_runtime': 237.3217, 'train_samples_per_second': 16.408, 'train_steps_per_second': 2.06, 'total_flos': 256145511688704.0, 'train_loss': 0.40751098757628773, 'epoch': 3.0})

In [37]:
preds_output = trainer.predict(val_dataset)

y_pred = np.argmax(preds_output.predictions, axis=1)
y_true = np.array(preds_output.label_ids)

print(classification_report(y_true, y_pred))

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.80      0.99      0.89       249
           3       0.79      0.25      0.38        59
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00        10

    accuracy                           0.80       325
   macro avg       0.32      0.25      0.25       325
weighted avg       0.76      0.80      0.75       325



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
